In [1]:
# Import key librarys

import pandas as pd
import numpy as np

import plotly.express as px

import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

In [2]:
# Setup Chart Studio
username = 'mrcrookes' # your username
api_key = 'ZgEVmRwdEN7u8cqkga9A' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [35]:
# Define level
level = 'Advanced_Higher'
level_label = level.replace('_', ' ')

# Import data
ind_df = pd.read_excel("attainment-statistics-(august)-2022-provisional-centre-independent.ods", 
                       engine = "odf", sheet_name = level, skiprows = 2)

state_df = pd.read_excel("attainment-statistics-(august)-2022-provisional-centre-education-authority.ods", 
                        engine = "odf", sheet_name = level, skiprows = 2)

In [36]:
def tidy_up_1(df):
    
    # Remove spaces and replace with '_'
    df.columns = df.columns.str.replace(' ','_')

    #df.drop(df[df.Grade_A_Count_2022 == '[z]'].index, inplace=True)
    df = df.replace('[z]', np.nan)
    df = df.replace('[c]', np.nan)
    df = df.replace('[low]', np.nan)

    # Remove last row
    df = df.drop(df.index[-1])

    # Remove columns with Percentage in the heading
    df = df[df.columns.drop(list(df.filter(regex='Percentage')))]
    
    return df
    
ind_df = tidy_up_1(ind_df)
state_df = tidy_up_1(state_df)

In [37]:
def tidy_up_2(df):

    # Create a list of years
    years = ['2022', '2021', '2020', '2019', '2018']

    # Sort out column titles
    for year in years:
        df = df.rename({'Grade_A_Count_' + year: 'As_' + year}, axis=1)
        df = df.rename({'Grades_A-B_Count_' + year: 'A-Bs_' + year}, axis=1)
        df = df.rename({'Grades_A-C_Count_' + year: 'A-Cs_' + year}, axis=1)
        df = df.rename({'Grades_A-D_Count_' + year: 'A-Ds_' + year}, axis=1)
        df = df.rename({'No_Award_Count_' + year: 'NAs_' + year}, axis=1)

    # Calculate number of Bs, Cs and Ds
    for year in years:
        df['Bs_' + year] = df['A-Bs_' + year] - df['As_' + year]
        df['Cs_' + year] = df['A-Cs_' + year] - df['A-Bs_' + year]
        df['Ds_' + year] = df['A-Ds_' + year] - df['A-Cs_' + year]

    # Remove original columns
    for year in years:
        df.drop('A-Bs_' + year, axis=1, inplace=True)
        df.drop('A-Cs_' + year, axis=1, inplace=True)
        df.drop('A-Ds_' + year, axis=1, inplace=True)
        
    return df
        
ind_df = tidy_up_2(ind_df)
state_df = tidy_up_2(state_df)

In [38]:
# Convert into long format

def tolong(df):
    ldf = pd.melt(df, id_vars = ['Subject'], var_name = 'abc', value_name ='Count')
    ldf['Grades'] = ldf['abc'].apply(lambda x: (x.rsplit('_', 1))[0])
    ldf['Year'] = ldf.pop('abc').apply(lambda x: (x.split('_'))[-1])
    
    return ldf
    
ind_ldf = tolong(ind_df)
state_ldf = tolong(state_df)

ind_ldf['School'] = 'Ind'
state_ldf['School'] = 'State'

ind_ldf.head()

,Subject,Count,Grades,Year,School
0,Accounting,10.0,As,2022,Ind
1,Art and Design (Design),40.0,As,2022,Ind
2,Art and Design (Expressive),65.0,As,2022,Ind
3,Biology,200.0,As,2022,Ind
4,Business Management,90.0,As,2022,Ind


In [39]:
# Merge together two df
#dfc = pd.merge(state_ldf, ind_ldf, on = ['Subject', 'Grades', 'Year', 'Count'])
dfc = pd.concat([state_ldf, ind_ldf])

dfc.head()

,Subject,Count,Grades,Year,School
0,Accounting,15.0,As,2022,State
1,Art and Design (Design),185.0,As,2022,State
2,Art and Design (Expressive),410.0,As,2022,State
3,Biology,570.0,As,2022,State
4,Business Management,110.0,As,2022,State


In [40]:
dfc = dfc[dfc.Grades == 'Entries']
dfc.dropna(inplace=True)

dfc.head()

,Subject,Count,Grades,Year,School
70,Accounting,45.0,Entries,2022,State
71,Art and Design (Design),605.0,Entries,2022,State
72,Art and Design (Expressive),1185.0,Entries,2022,State
73,Biology,2675.0,Entries,2022,State
74,Business Management,535.0,Entries,2022,State


In [41]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319 entries, 70 to 494
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Subject  319 non-null    object 
 1   Count    319 non-null    float64
 2   Grades   319 non-null    object 
 3   Year     319 non-null    object 
 4   School   319 non-null    object 
dtypes: float64(1), object(4)
memory usage: 15.0+ KB


In [42]:
# Plot graph

# Create a list of subjects in alphabetical order
subjects = sorted(dfc['Subject'].unique())
years = sorted(dfc['Year'].unique())

fig = px.line(dfc, x = "Year", y = "Count", 
                color = "Subject",
                facet_row = 'School',
                width = 800,
                height = 1150,
                category_orders = {'Subject' : subjects, 'Year' : years},
                labels = {"perc": "Percentage"},
                title = 'Entries per subject from 2018 - 2022 at ' + level_label
                )

fig.update_traces(mode = "markers+lines")

fig.update_yaxes(matches=None, rangemode = "tozero")

fig.write_html('./graphs/entries_line_' + level + '.html')

fig.show()

In [43]:
# Upload to Chart Studio
py.plot(fig, filename = 'entries_line_' + level, auto_open=True)

# Output html embed code
#tls.get_embed('https://plotly.com/~mrcrookes/14/') #change to your url

'https://plotly.com/~mrcrookes/35/'